<h3>讀取套件</h3>

In [1]:
import pandas
import os

<h3>建立資料路徑</h3>

In [2]:
data_path = 'C:\\Users\\Jerry\\Desktop\\Thesis\\data\\Raw'

<h3>建立分群變數</h3>

In [4]:
region = {
    'A': ['台北一', '台北二', '板橋區', '三重區'],
    #'B': ['桃園', '台中','豐原', '高雄市','鳳山', '屏東'],
    #'C': ['台北一', '台北二', '板橋', '三重','桃園', '台中','豐原', '高雄市','鳳山', '屏東']
}

market_categoty = '蔬菜產品日交易行情'
col_names=['日期', '市場', '產品', '上價', '中價', '下價', '均價','增減%', '交易量']

<h3>萃取資料</h3>

In [5]:
def get_data(region_group, region_id):
    global data_path, col_names
    
    excel_file = '%s/%s/%s%s.xls' % (data_path, region_group, region_id, market_categoty)
    if not os.path.exists(excel_file):
        print(excel_file)
        return None
    
    market = pandas.read_excel(
        excel_file, header=4, skipfooter=1, 
        names=col_names, usecols='A:I').reset_index().drop(columns=['index'])
    
    return market

<h3>重排資料表</h3>

In [6]:
def adj_data(dataset):
    new_date_list = []
    dist_id_list = []
    dist_name_list = []
    product_id_list = []
    product_name_list = []
    product_type_list = []
    
    for i in range(dataset.shape[0]):
        row = dataset.iloc[i]
        
        date_parts = row['日期'].split('/')
        new_date = '%04d-%s' % (int(date_parts[0]) + 1911, '-'.join(date_parts[1:]))
        new_date_list.append(new_date)
        
        dist_parts = row['市場'].split()
        dist_id_list.append(dist_parts[0])
        dist_name_list.append(dist_parts[1])
        
        product_parts = row['產品'].split()
        if len(product_parts) == 2:
            product_id_list.append(product_parts[0])
            product_name_list.append(product_parts[1])
            product_type_list.append('')
        else:    
            product_id_list.append(product_parts[0])
            product_name_list.append(product_parts[1])
            product_type_list.append(product_parts[2])
        
    dataset['日期'] = new_date_list
    dataset['市場'] = dist_id_list
    dataset['地區'] = dist_name_list
    dataset['產品'] = product_id_list
    dataset['名稱'] = product_name_list
    dataset['品名'] = product_type_list
    
    return dataset[['市場', '日期', '產品', '上價', '中價', '下價', '均價', '地區', '名稱', '品名', '交易量']]

<h3>輸出資料CSV</h3>

In [8]:
data_path_2 = 'C:\\Users\\Jerry\\Desktop\\Thesis\\data\\Transformation'

#for region_group in ['A', 'B', 'C']:
for region_group in ['A']:
    combined_dataset = None

    for region_id in region[region_group]:
        df = get_data(region_group, region_id)
        if df is not None:
            df2 = adj_data(df)
            df2.to_csv('%s/%s/%s-%s.csv' % (data_path_2, region_group, market_categoty, region_id), index=False)

<h3>首次-合併資料和輸出產品對照表</h3>

In [9]:
data_path_3 = 'C:\\Users\\Jerry\\Desktop\\Thesis\\data\\Dataset'

#for region_group in ['A', 'B', 'C']:
for region_group in ['A']:
    combined_dataset = None
    
    for region_id in region[region_group]:
        csv_file = '%s/%s/%s-%s.csv' % (data_path_2, region_group, market_categoty, region_id)
        if not os.path.exists(csv_file):
            print(csv_file)
            continue
            
        df = pandas.read_csv(csv_file, header=0)
        
        df_data = df[['市場', '日期', '產品', '上價', '中價', '下價', '均價', '交易量']].copy()
        df_data.to_csv('%s/%s/%s-%s-2.csv' % (data_path_3, region_group, market_categoty, region_id), index=False)

        if combined_dataset is None:
            combined_dataset = df
        else:
            combined_dataset = pandas.concat([combined_dataset, df], axis=0)
            
    df_data = combined_dataset[['市場', '日期', '產品', '上價', '中價', '下價', '均價', '交易量']].copy()
    df_data.to_csv('%s/%s/%s-DS.csv' % (data_path_3, region_group, market_categoty), index=False)
    
    df_data = combined_dataset[['產品', '名稱','品名']].drop_duplicates()
    df_data.to_csv('%s/%s/產品名稱對照表.csv' % (data_path_3, region_group), index=False)   

<h3>累加-新的資訊進去原有資料集</h3>

In [ ]:
data_path_3 = 'C:\\Users\\Jerry\\Desktop\\Thesis\\data\\Dataset'

#for region_group in ['A', 'B', 'C']:
for region_group in ['A']:
    combined_dataset = None

    for region_id in region[region_group]:
        df = get_data(region_group, region_id).dropna()
        if df is not None:
            df2 = adj_data(df)
            if combined_dataset is None:
                combined_dataset = df2
            else:
                combined_dataset = pandas.concat([combined_dataset, df2], axis=0)            
            
    df_data = combined_dataset[['市場', '日期', '產品', '上價', '中價', '下價', '均價', '交易量']].copy()
    dataset = pandas.read_csv('%s/%s/%s-DS.csv' % (data_path_3, region_group, market_categoty), header=0)
    DS_update = pandas.concat([dataset, df_data], axis=0).drop_duplicates()
    DS_update.to_csv('%s/%s/%s-DS_updated.csv' % (data_path_3, region_group, market_categoty), index=False)
    
    df_data_1 = combined_dataset[['產品', '名稱','品名']].copy()
    dataset_1 = pandas.read_csv('%s/%s/產品名稱對照表.csv' % (data_path_3, region_group), header=0).fillna('')
    DS_update_1 = pandas.concat([dataset_1, df_data_1], axis=0).drop_duplicates()
    DS_update_1.to_csv('%s/%s/產品名稱對照表_updated.csv' % (data_path_3, region_group), index=False) 